# Limpieza de los datos de la Base de datos `bookmaker.db`

Antes de empezar a desarrollar la limpieza de los datos, hemos seleccionado como Kernel el entorno que hemos creado previamente

In [1]:
%pip install numpy
%pip install matplotlib
%pip install pandas

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.



In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sqlite3

C:\Users\mglez\AppData\Local\Temp\ipykernel_8148\2253275120.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Ahora vamos a desglosar la base de datos en 6 tabblas, separadas unas de otras

In [3]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect('bookmaker.db')

In [4]:
conn.commit()

clientes = pd.read_sql_query("SELECT * FROM clientes", conn)
empresa = pd.read_sql_query("SELECT * FROM empresa", conn)
equipos = pd.read_sql_query("SELECT * FROM equipos", conn)
partidos = pd.read_sql_query("SELECT * FROM partidos", conn)
apuestas = pd.read_sql_query("SELECT * FROM apuestas", conn)
cuotas = pd.read_sql_query("SELECT * FROM cuotas", conn)

tablas = [clientes, empresa, equipos, partidos, apuestas, cuotas]


Visualizamos que todas las tablas contienen los datos de la base de datos

In [10]:
clientes.head()

,id,nombre,apellido,email,activado
0,1,Lope,Perales,pascualperera@vila.es,1
1,2,Isaura,Canet,gmerino@rocamora.es,1
2,3,Olga,Chaves,amayaprats@llado.com,1
3,4,Celestino,Hoyos,chedelgado@moraleda-pintor.com,1
4,5,Miguel,Milla,inigoprat@gmail.com,1


In [11]:
empresa.head()

,id,razon_social,email,activado


In [12]:
equipos.head()

,id,nombre,escudo,pais,puntaje,activado
0,1,Ajax,ajax.png,Paises Bajos,14.0,1
1,2,Atalanta,atalanta.png,Italia,6.0,1
2,3,Atlético,atletico.png,España,11.0,1
3,4,Barcelona,barcelona.png,España,17.0,1
4,5,Bayern,bayern.png,Alemania,16.0,1


In [13]:
partidos.head()

,id,fecha,equipo_local,equipo_visistante,finalizado,ganador
0,1,2021-12-15 00:51:23.181740,29,2,0,None
1,2,2021-12-15 00:51:23.181740,31,19,0,None
2,3,2021-12-15 00:51:23.181740,25,24,0,None
3,4,2021-12-15 00:51:23.181740,11,6,0,None
4,5,2021-12-15 00:51:23.181740,4,5,0,None


In [14]:
apuestas.head()

,id,fecha,monto,equipo_ganador_id,partido,cliente,ganacia
0,1,2021-12-15 01:08:30.211674,300.0,2.0,1,88,891.0
1,2,2021-12-15 01:08:30.211674,300.0,29.0,1,88,891.0
2,3,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0
3,4,2021-12-15 01:35:34.965516,300.0,29.0,1,88,891.0
4,5,2021-12-15 01:35:34.965516,300.0,2.0,1,88,891.0


In [15]:
cuotas.head()

,id,local,empate,visitante,partido_id
0,1,1.40,2.97,5.80,1
1,2,5.73,5.08,3.86,2
2,3,5.45,5.99,5.34,3
3,4,1.95,5.08,4.78,4
4,5,4.44,5.99,4.61,5


Como podemos ver, todas las tablas contienen datos menos la de `empresa`, por lo que la de empresa no va a ser tratada